In [1]:
from riotwatcher import LolWatcher, ApiError
import pandas as pd

API_KEY = 'RGAPI-8451d9b1-4ee5-4a02-a8ae-da4dfee0aa7d'

In [2]:
my_region = 'la2'
lol_watcher = LolWatcher(API_KEY)
me = lol_watcher.summoner.by_name(my_region, 'srTe')
print(me)

{'id': 'lWi5HaaehFeaGAfBcTt0rC1ZwbGADdfzMufodzqWLxti', 'accountId': '7GCep_kNbjnlDwUs_YzG2m72hscya8h4mONfTirSuY4', 'puuid': 'Tbchv-3XdognbbbeZmtdB-bWEEu4dFuug9aq1lc1vJIDWtcZOv4YHP6a5CnfHO8yqUgR3KMACSt0FA', 'name': 'srTe', 'profileIconId': 3379, 'revisionDate': 1622756022000, 'summonerLevel': 165}


In [5]:
my_ranked_stats = lol_watcher.league.by_summoner(my_region, me['id'])
print(my_ranked_stats)

my_matches = lol_watcher.match.matchlist_by_account(my_region, me['accountId'])
last_match = my_matches['matches'][99]
match_detail = lol_watcher.match.by_id(my_region, last_match['gameId'])
print(last_match)
print(len(my_matches['matches']))

[{'leagueId': '889fff5e-4a8d-4d1f-ad95-ff0e3ce46ea5', 'queueType': 'RANKED_FLEX_SR', 'tier': 'SILVER', 'rank': 'I', 'summonerId': 'lWi5HaaehFeaGAfBcTt0rC1ZwbGADdfzMufodzqWLxti', 'summonerName': 'srTe', 'leaguePoints': 37, 'wins': 37, 'losses': 32, 'veteran': False, 'inactive': False, 'freshBlood': False, 'hotStreak': False}]
{'platformId': 'LA2', 'gameId': 995676246, 'champion': 119, 'queue': 400, 'season': 13, 'timestamp': 1615251140065, 'role': 'DUO', 'lane': 'MID'}
100


In [7]:
participants = []
for row in match_detail['participants']:
    participants_row = {}
    participants_row['champion'] = row['championId']
    participants_row['spell1'] = row['spell1Id']
    participants_row['spell2'] = row['spell2Id']
    participants_row['win'] = row['stats']['win']
    participants_row['kills'] = row['stats']['kills']
    participants_row['assists'] = row['stats']['assists']
    participants_row['deaths'] = row['stats']['deaths']
    participants_row['totalDamageDealt'] = row['stats']['totalDamageDealt']
    participants_row['goldEarned'] = row['stats']['goldEarned']
    participants_row['champLevel'] = row['stats']['champLevel']
    participants_row['totalMinionsKilled'] = row['stats']['totalMinionsKilled']
    participants_row['item0'] = row['stats']['item0']
    participants_row['item1'] = row['stats']['item1']
    participants.append(participants_row)
df = pd.DataFrame(participants)
df

,champion,spell1,spell2,win,kills,assists,deaths,totalDamageDealt,goldEarned,champLevel,totalMinionsKilled,item0,item1
0,33,4,32,False,5,16,14,31566,11825,18,19,3111,3067
1,429,3,4,False,6,9,12,80396,13811,17,71,3153,3072
2,876,32,4,False,7,10,16,53722,12815,16,41,6653,3157
3,202,21,4,False,9,10,12,89089,13281,18,61,6676,6671
4,517,4,14,False,5,12,13,34428,12108,17,20,3111,6656
5,42,21,4,True,24,26,6,97800,16748,18,52,3042,3508
6,9,4,14,True,22,28,8,60065,16094,18,39,3157,6653
7,117,4,14,True,6,44,5,13109,12921,18,8,6617,3070
8,53,21,4,True,1,24,7,36972,11445,18,28,6656,3158
9,10,13,4,True,14,35,6,69355,15141,18,61,6672,3153
